<a href="https://colab.research.google.com/github/naphataim-glitch/Big/blob/main/Lab_class_4_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


*ชื่อสมาชิกกลุ่ม*
นายณภัทร อิ่มอ่ำ 6704800039

นายพุฒิภัทร พันธ์ดี 6704800037

นางสาวภคพร สุภาพ 6704800015

นายภัควัตร สุริวงษ์ 6704800030

In [1]:
#!/bin/bash
!curl -L -o ./brazilian-ecommerce.zip\
  https://www.kaggle.com/api/v1/datasets/download/olistbr/brazilian-ecommerce

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 42.6M  100 42.6M    0     0  84.5M      0 --:--:-- --:--:-- --:--:--  299M


In [ ]:
!unzip /content/brazilian-ecommerce.zip

Archive:  /content/brazilian-ecommerce.zip
  inflating: olist_customers_dataset.csv  
  inflating: olist_geolocation_dataset.csv  
  inflating: olist_order_items_dataset.csv  
  inflating: olist_order_payments_dataset.csv  
  inflating: olist_order_reviews_dataset.csv  
  inflating: olist_orders_dataset.csv  
  inflating: olist_products_dataset.csv  
  inflating: olist_sellers_dataset.csv  
  inflating: product_category_name_translation.csv  


### 🛠️ Task 1: Load & Clean Data

1. อ่านไฟล์ CSV ทั้ง 4 ไฟล์เข้าสู่ Spark DataFrame
2. ตรวจสอบ Schema และจัดการ Type (ถ้าจำเป็น)
3. สร้าง Temp View เพื่อเตรียมพร้อมสำหรับ SQL

In [7]:
# 1.1 อ่านไฟล์ CSV ทั้ง 4 ไฟล์
df_orders = spark.read.csv("olist_orders_dataset.csv", header=True, inferSchema=True)
df_items = spark.read.csv("olist_order_items_dataset.csv", header=True, inferSchema=True)
df_products = spark.read.csv("olist_products_dataset.csv", header=True, inferSchema=True)
df_customers = spark.read.csv("olist_customers_dataset.csv", header=True, inferSchema=True)

# 1.2 ตรวจสอบ Schema (ดูว่า Type ตรงไหม)
df_orders.printSchema()

# 1.3 สร้าง Temp View เพื่อใช้ SQL (ถ้าต้องการ)
df_orders.createOrReplaceTempView("orders")
df_items.createOrReplaceTempView("items")
df_products.createOrReplaceTempView("products")
df_customers.createOrReplaceTempView("customers")

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



In [8]:
# สร้าง df_master โดยการ Join ตารางตามลำดับ
df_master = df_orders.join(df_items, on="order_id", how="inner") \
                     .join(df_products, on="product_id", how="inner") \
                     .join(df_customers, on="customer_id", how="inner")

# ตรวจสอบผลลัพธ์ (Tip: ดูว่าจำนวนแถวสมเหตุสมผลไหม)
print(f"จำนวนแถวทั้งหมดหลัง Join: {df_master.count():,}")
df_master.select("order_id", "product_category_name", "price", "customer_city").show(5)

จำนวนแถวทั้งหมดหลัง Join: 112,650
+--------------------+---------------------+-----+--------------------+
|            order_id|product_category_name|price|       customer_city|
+--------------------+---------------------+-----+--------------------+
|00010242fe8c5a6d1...|           cool_stuff| 58.9|campos dos goytac...|
|00018f77f2f0320c5...|             pet_shop|239.9|     santa fe do sul|
|000229ec398224ef6...|     moveis_decoracao|199.0|       para de minas|
|00024acbcdf0a6daa...|           perfumaria|12.99|             atibaia|
|00042b26cf59d7ce6...|   ferramentas_jardim|199.9|     varzea paulista|
+--------------------+---------------------+-----+--------------------+
only showing top 5 rows


### 🔗 Task 2: Join Data

จงสร้าง `df_master` โดยการ Join ตารางดังนี้:
1. `orders` JOIN `items` (ON order_id)
2. JOIN `products` (ON product_id)
3. JOIN `customers` (ON customer_id)

> **Tip:** ตรวจสอบจำนวนแถวหลัง Join ว่าเพิ่มขึ้นหรือลดลงผิดปกติหรือไม่


### 📊 Task 3: Analytics

ตอบคำถามต่อไปนี้ (เลือกใช้ SQL หรือ PySpark ก็ได้):

1. **Top Products:** สินค้าหมวดไหน (`product_category_name`) มียอดขายรวม (price) สูงที่สุด 5 อันดับแรก?
2. **Top Cities:** เมืองไหน (`customer_city`) มีจำนวนคำสั่งซื้อมากที่สุด 5 อันดับแรก?
3. **(Optional) Monthly Sales:** ยอดขายรวมแต่ละเดือนเป็นอย่างไร? (แนวโน้ม)


In [6]:
!unzip /content/brazilian-ecommerce.zip -d /content/



Archive:  /content/brazilian-ecommerce.zip
replace /content/olist_customers_dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 